In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import time
import datetime as dt

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### READ IN CANARY DATA

In [4]:
canary_log = pq.read_table('../data/CANARY/CANARY/ACTIVITY_LOG').to_pandas()
canary_ps  = pq.read_table('../data/CANARY/CANARY/PERSON_SUMMARY').to_pandas()


### READ IN BIOMETRIC DATA

In [5]:
biometric = pq.read_table('../data/BIOMETRICS').to_pandas()

### Merge Canary and Biometrics

In [6]:
merged = canary_ps.merge(biometric, on = 'PERS_ID', how = 'inner')

### Ensure dates are in the same Datetime Date format

In [7]:
merged['EVENT_DATE'] = pd.to_datetime(merged.EVENT_DATE).dt.date

In [8]:
merged.head()

PERS_ID       MBR_ID GROUPNUMBER               GROUPNAME       LOB                                          FILE_NAME   FILE_DATE  N_MILESTONES_ACHIEVED REGISTER_DATE LAST_ACTIVITY_DATE LAST_COMPLETED_MILESTONE  EVENT_DATE             DATE_TYPE BIOMETRIC_CATEGORY EVENT_CODE CODING_STANDARD      SOURCE        RESULT_VALUE
0  2638452  M0006106300       37330  Medicare Advantage 175  Medicare  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07                      1    2020-01-26         2020-01-26            REGISTER_DATE  2019-10-28             CALC_DATE                BMI       None            None  CALCULATED   24.80655324511657
1  2638452  M0006106300       37330  Medicare Advantage 175  Medicare  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07                      1    2020-01-26         2020-01-26            REGISTER_DATE  2018-08-03  OBSERVATION_DATETIME             Weight     3141-9                       HIXNY  160.93725999999998
2  2638452  M0006106300       37330  Medicare Advantage 175  Medicare  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07                      1    2020-01-26         2020-01-26            REGISTER_DATE  2018-08-30             CALC_DATE                BMI       None            None  CALCULATED  24.434210151237654
3  2638452  M0006106300       37330  Medicare Advantage 175  Medicare  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07                      1    2020-01-26         2020-01-26            REGISTER_DATE  2018-08-02       result_datetime           Troponin    10839-9              LN       HIXNY   9.119999885559082
4  2638452  M0006106300       37330  Medicare Advantage 175  Medicare  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07                      1    2020-01-26         2020-01-26            REGISTER_DATE  2018-08-02             CALC_DATE                BMI       None            None  CALCULATED  24.325276955299785

### Filter Data by category

In [9]:
cat_filt = merged.BIOMETRIC_CATEGORY == 'BMI'
merged_cat = merged[cat_filt]

### Drop Duplicate Ids, Event Dates

In [10]:
merged_drop = merged_cat.drop_duplicates(subset = ['PERS_ID', 'EVENT_DATE'])

In [11]:
def bmi_dates(group):
    min_date = group['EVENT_DATE'].min()
    max_date = group['EVENT_DATE'].max()
    total_CAT = len(group['EVENT_DATE'])
    missing = ((max_date - min_date).days // 30) - total_CAT +1
    span    = total_CAT + missing
    reg_date = group.REGISTER_DATE[0]
    timedelta = dt.timedelta(days = 8*30)
    check_from = reg_date - timedelta
    
    return pd.DataFrame([[reg_date, check_from, min_date, max_date, total_CAT, missing, span]], columns = ['REGISTER_DATE', 'CHECK_FROM','MIN_DATE_BMI', 'MAX_DATE_BMI', 'TOTAL_MEAS', 'MISSING', 'SPAN'])

### Sort by PERSON IDS and EVENT DATES

In [12]:
merged_drop = merged_drop.sort_values(by = ['PERS_ID', 'EVENT_DATE']).reset_index(drop = True)

In [13]:
events = merged_drop.groupby('PERS_ID',squeeze = True).apply(bmi_dates)
events.index = events.index.droplevel(1)

In [14]:
events

REGISTER_DATE  CHECK_FROM MIN_DATE_BMI MAX_DATE_BMI  TOTAL_MEAS  MISSING  SPAN
PERS_ID                                                                               
2602482    2019-08-09  2018-12-12   2018-03-09   2020-03-16          13       12    25
2603072    2020-01-01  2019-05-06   2006-05-03   2020-08-04          34      140   174
2619378    2019-10-31  2019-03-05   2007-01-23   2020-09-07          49      117   166
2621187    2020-01-23  2019-05-28   2016-05-03   2020-06-25          12       39    51
2623521    2019-03-20  2018-07-23   2012-11-13   2020-09-08          63       33    96
...               ...         ...          ...          ...         ...      ...   ...
5025690    2020-07-17  2019-11-20   2015-05-04   2020-08-05          12       53    65
5028397    2020-05-06  2019-09-09   2020-01-28   2020-01-28           1        0     1
5030333    2020-04-24  2019-08-28   2020-02-13   2020-02-13           1        0     1
5036084    2020-05-29  2019-10-02   2011-07-08   2020-09-02          47       65   112
5036817    2020-05-27  2019-09-30   2006-12-19   2020-06-19          54      111   165

[1182 rows x 7 columns]

### We will take the first CATEGORY At most 5 Months Before the Registration Date in Canary program  as a Reference and create the binary Outcome Variable

In [15]:
merged_drop.PERS_ID.unique().shape, events.index.shape

((1182,), (1182,))

In [16]:
indexes_to_drop  = []

ids = events.index.unique()

for idi in ids:
    date_to_check = events.loc[idi, 'CHECK_FROM']
    groupi = merged_drop[merged_drop.PERS_ID == idi]
    indx = groupi[groupi.EVENT_DATE < events.loc[idi, 'CHECK_FROM']].index.tolist()
    indexes_to_drop.extend(indx)

### DROP ROWS WITH CATEGORYs MEASURED AT LEAST 8 MONTHS BEFORE CANARY REGISTARTION

In [17]:
merged2  = merged_drop.drop(index = indexes_to_drop).reset_index(drop = True)

In [18]:
merged2.head()

PERS_ID       MBR_ID GROUPNUMBER       GROUPNAME LOB                                          FILE_NAME   FILE_DATE  N_MILESTONES_ACHIEVED REGISTER_DATE LAST_ACTIVITY_DATE LAST_COMPLETED_MILESTONE  EVENT_DATE             DATE_TYPE BIOMETRIC_CATEGORY EVENT_CODE CODING_STANDARD      SOURCE        RESULT_VALUE
0  2602482  6F995322100       35938  365 LifePoints  LG  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07                      1    2019-08-09         2019-08-09            REGISTER_DATE  2019-04-21             CALC_DATE                BMI       None            None  CALCULATED  27.583445310400432
1  2602482  6F995322100       35938  365 LifePoints  LG  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07                      1    2019-08-09         2019-08-09            REGISTER_DATE  2019-07-16             CALC_DATE                BMI       None            None  CALCULATED   27.77237301800591
2  2602482  6F995322100       35938  365 LifePoints  LG  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07                      1    2019-08-09         2019-08-09            REGISTER_DATE  2019-09-15  OBSERVATION_DATETIME                BMI    39156-5              LN       HIXNY  27.799999237060547
3  2602482  6F995322100       35938  365 LifePoints  LG  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07                      1    2019-08-09         2019-08-09            REGISTER_DATE  2019-09-16                SVC_DT                BMI       None            None  HEDIS Supp                27.8
4  2602482  6F995322100       35938  365 LifePoints  LG  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07                      1    2019-08-09         2019-08-09            REGISTER_DATE  2019-09-30  OBSERVATION_DATETIME                BMI    39156-5              LN       HIXNY  27.100000381469727

### Find Reference CATEGORY value

In [19]:
cat_ref = merged2.groupby('PERS_ID').apply(lambda x: x.RESULT_VALUE[0])


### GET INDICATOR VARIABLES FOR CATEGORY 1 CAT_VALUE DECREASED 0 CAT_VALUE INCREASED

In [20]:
def cat_ind(group):
    new_group = group[['PERS_ID','EVENT_DATE', 'REGISTER_DATE', 'LAST_ACTIVITY_DATE', 'BIOMETRIC_CATEGORY', 'RESULT_VALUE']]
    idi = group.PERS_ID[0]
    #get the reference value
    ref_value = cat_ref[idi]
    new_group['BMI_IND'] = (new_group['RESULT_VALUE'] <  ref_value).astype(int)
    return new_group
    
    

In [21]:
final = merged2.groupby('PERS_ID').apply(cat_ind)

In [22]:
final2 = final.copy()


In [23]:
final2.head(), final2.shape

(   PERS_ID  EVENT_DATE REGISTER_DATE LAST_ACTIVITY_DATE BIOMETRIC_CATEGORY        RESULT_VALUE  BMI_IND
 0  2602482  2019-04-21    2019-08-09         2019-08-09                BMI  27.583445310400432        0
 1  2602482  2019-07-16    2019-08-09         2019-08-09                BMI   27.77237301800591        0
 2  2602482  2019-09-15    2019-08-09         2019-08-09                BMI  27.799999237060547        0
 3  2602482  2019-09-16    2019-08-09         2019-08-09                BMI                27.8        0
 4  2602482  2019-09-30    2019-08-09         2019-08-09                BMI  27.100000381469727        1,
 (6914, 7))

### Transform it in an approrpiate format for survival analysis

In [24]:
def duration_event(group):
    indx = np.where(group['BMI_IND'] == 1)[0]
    if len(indx) == 0:
        group['DURATION'] = len(group)
        group['EVENT']    = 0
        return group
    else:
        group['DURATION'] = indx[0] + 1
        group['EVENT']    = 1
        return group.iloc[0:indx[0]+1]

In [25]:
final3 = final2.groupby('PERS_ID').apply(duration_event).reset_index(drop = True)


### Load matching ids

In [26]:
import pickle
mids = pickle.load(open('../temp/matching_ids.p', 'rb'))

In [27]:
control , treated = mids['control'], mids['treated']

In [28]:
control_d, treated_d = final3[final3.PERS_ID.isin(control)], final3[final3.PERS_ID.isin(treated)]

In [29]:
control_d.drop_duplicates(subset = 'PERS_ID', inplace = True)

Empty DataFrame
Columns: [PERS_ID, EVENT_DATE, REGISTER_DATE, LAST_ACTIVITY_DATE, BIOMETRIC_CATEGORY, RESULT_VALUE, BMI_IND, DURATION, EVENT]
Index: []

In [30]:
treated_d.drop_duplicates(subset = 'PERS_ID', inplace = True)

Empty DataFrame
Columns: [PERS_ID, EVENT_DATE, REGISTER_DATE, LAST_ACTIVITY_DATE, BIOMETRIC_CATEGORY, RESULT_VALUE, BMI_IND, DURATION, EVENT]
Index: []

In [31]:
duration_t,  events_t = treated_d['DURATION'].tolist(), treated_d['EVENT'].tolist()
duration_c,  events_c = control_d['DURATION'].tolist(), control_d['EVENT'].tolist()

In [32]:
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt
kmf = KaplanMeierFitter()
kmf.fit(duration_t, event_observed=events_t, label = 'Treated')  
ax = kmf.plot()
kmf2 = KaplanMeierFitter()
kmf2.fit(duration_c, event_observed=events_c, label = 'Control')  
kmf2.plot(ax = ax)
plt.show()

ValueError: Values must be numeric: no strings, datetimes, objects, etc.